# Poisson non-linear problem

Let's now try to solve a non-linear Poisson (test) problem

The PDE will have the following form: -div(q(u),grad(u))=f in Omega

(with the following Dirichelet BC: u=uD)

The problem is non linear since: q(u)= 1 + u**2

In [2]:
import ufl
import numpy

from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import mesh, fem, io, nls, log
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver

def q(u):
    return 1 + u**2


# define the domain 
domain=mesh.create_unit_square(MPI.COMM_WORLD,10,10)

# identify the coordinates of the mesh nodes 
x=ufl.SpatialCoordinate(domain)
u_ufl=1+x[0]+2*x[1] # this is the exact solution that I will use to create the Dirichelet BC and also f 
f=-ufl.div(q(u_ufl)*ufl.grad(u_ufl))

In [3]:
V = fem.functionspace(domain, ("Lagrange", 1))
def u_exact(x): 
    return eval(str(u_ufl))

In [4]:
u_D = fem.Function(V)
u_D.interpolate(u_exact)

# now I define the BC 
# to do this I will have to define the facets on the boundary 
# and the dofs on the boundary 
fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(domain, fdim, lambda x: numpy.full(x.shape[1], True, dtype=bool))
bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, boundary_facets))

Now I can define the variational problem

(Since the problem is non-linear I will not have to define a trial function (u), but simply define a Function (uh) which will be the unknown of our problem)

In [5]:
uh = fem.Function(V)
v = ufl.TestFunction(V)
F = q(uh) * ufl.dot(ufl.grad(uh), ufl.grad(v)) * ufl.dx - f * v * ufl.dx 
# F(u,v) should be =0, so we can treat F as a residual!!!!

# Newton method

In the case of non-linear systems we will need to minimize the residual F

To do this we will use techniques such as:

-Newton's method

In the Newton method procedure we will need to compute Jacobian matrices

DOLFINx provides the 'NonLinearProblem' function that implements these methods!

In [6]:
problem=NonlinearProblem(F,uh,bcs=[bc])

# In this way I have defined the non-linear problem!    

As a solver for the non-linear problem we will therefore use Newton's method

In [7]:
solver=NewtonSolver(MPI.COMM_WORLD,problem)
solver.convergence_criterion='incremental'
solver.rtol=1e-6 # set the tolerance to be met in the convergence criterion
solver.report=True # ensures that during the solution important information is reported on each loop performed 

In [8]:
# ksp = solver.krylov_solver
# opts = PETSc.Options()
# option_prefix = ksp.getOptionsPrefix()


# opts[f"{option_prefix}ksp_type"] = "gmres"
# opts[f"{option_prefix}ksp_rtol"] = 1.0e-8
# opts[f"{option_prefix}pc_type"] = "hypre"
# opts[f"{option_prefix}pc_hypre_type"] = "boomeramg"
# opts[f"{option_prefix}pc_hypre_boomeramg_max_iter"] = 1
# opts[f"{option_prefix}pc_hypre_boomeramg_cycle_type"] = "v"
# ksp.setFromOptions()

Now we are able to solve the problem

In [9]:
log.set_log_level(log.LogLevel.INFO) # Set the log level for dolfinx.log to INFO.
# This means that will be printed:
# details on Newton iterations,
# absolute and relative residuals,
# calls to the internal linear solver.

n, converged = solver.solve(uh) # start solving the non-linear problem
# outputs: 
# n<---the number of iterations 
# converged<---a boolean that tells us if convergence was reached

assert (converged) # an exception (AssertionError) will be raised if assert is not True!!!!!


print(f"Number of interations: {n:d}")

Number of interations: 8
[2025-09-01 17:38:06.569] [info] PETSc Krylov solver starting to solve system.
[2025-09-01 17:38:06.570] [info] PETSc Krylov solver starting to solve system.
[2025-09-01 17:38:06.570] [info] Newton iteration 2: r (abs) = 20.37916572199396 (tol = 1e-10), r (rel) = 0.9225323396539479 (tol = 1e-06)
[2025-09-01 17:38:06.570] [info] PETSc Krylov solver starting to solve system.
[2025-09-01 17:38:06.570] [info] Newton iteration 3: r (abs) = 6.9527130087520765 (tol = 1e-10), r (rel) = 0.3147382324873168 (tol = 1e-06)
[2025-09-01 17:38:06.570] [info] PETSc Krylov solver starting to solve system.
[2025-09-01 17:38:06.570] [info] Newton iteration 4: r (abs) = 2.9357037199767175 (tol = 1e-10), r (rel) = 0.13289462671173238 (tol = 1e-06)
[2025-09-01 17:38:06.570] [info] PETSc Krylov solver starting to solve system.
[2025-09-01 17:38:06.570] [info] Newton iteration 5: r (abs) = 0.7005897396309602 (tol = 1e-10), r (rel) = 0.03171458049147569 (tol = 1e-06)
[2025-09-01 17:38:0

We observe that the solver converges in only 8 iterations 

I am going to calculate:

-L2error

-Max error 

Taking into account the analytical (exact) solution

In [10]:
# Compute L2 error and error at nodes
V_ex = fem.functionspace(domain, ("Lagrange", 2))
u_ex = fem.Function(V_ex)
u_ex.interpolate(u_exact)

# Calculate the L2_error:
error_local = fem.assemble_scalar(fem.form((uh - u_ex)**2 * ufl.dx))
error_L2 = numpy.sqrt(domain.comm.allreduce(error_local, op=MPI.SUM))
if domain.comm.rank == 0:
    print(f"L2-error: {error_L2:.2e}")

# Calculate the max error in the mesh dofs 
error_max = domain.comm.allreduce(numpy.max(numpy.abs(uh.x.array - u_D.x.array)), op=MPI.MAX)
if domain.comm.rank == 0:
    print(f"Error_max: {error_max:.2e}")

[2025-09-01 17:38:06.578] [info] Checking required entities per dimension
[2025-09-01 17:38:06.578] [info] Cell type: 0 dofmap: 200x6
[2025-09-01 17:38:06.578] [info] Global index computation
[2025-09-01 17:38:06.578] [info] Got 2 index_maps
[2025-09-01 17:38:06.578] [info] Get global indices
L2-error: 4.79e-16
Error_max: 8.88e-16
